In [27]:
import torch
import os

from argparse import Namespace
from momentfm import MOMENTPipeline
from momentfm import MOMENT
from accelerate import Accelerator
from src.data.data_utils import *
from src.data.dataset import *
from torch.utils.data import Dataset, DataLoader

In [7]:
data_path = '/scratch/bbug/ayang1/datasets/lucas_fused/'
model_path = '/scratch/bbug/ayang1/experiments/saved/'
test_loader = DataLoader(CropTypeDataset(data_path, 'test'), batch_size=1, shuffle=False)

In [49]:
config = {
    "task_name": "classification", 
    "model_name": "MOMENT", 
    "transformer_type": "encoder_only", 
    "d_model": None, 
    "seq_len": 512, 
    "patch_len": 8, 
    "patch_stride_len": 8, 
    "device": "cpu", 
    "transformer_backbone": "google/flan-t5-large", 
    "n_channels": 7,
    "num_class": 12,
    "freeze_encoder": False,
    "freeze_embedder": False,
    "reduction": 'concat',
    "enable_gradient_checkpointing": False
    }

config = Namespace(**config)
m2 = MOMENT(config)

with open(os.path.join(model_path, 'benchmarkv2_30_checkpoint/pytorch_model/mp_rank_00_model_states.pt'), 'rb') as f:
    model = torch.load(f, map_location=torch.device('cpu'))
    

In [50]:
m2.load_state_dict(model['module'])

<All keys matched successfully>

In [51]:
example = test_loader.dataset[1123]
m2(example[0].unsqueeze(0), example[1].unsqueeze(0))

TimeseriesOutputs(forecast=None, anomaly_scores=None, logits=tensor([[11.6153, -6.3887,  1.9153, -1.6587, -7.7097, 17.5951, -7.1837, -5.4228,
         -2.3393, -2.2541, -6.8590, -8.5810]], grad_fn=<AddmmBackward0>), labels=None, input_mask=None, pretrain_mask=None, reconstruction=None, embeddings=tensor([[[ 0.1616, -0.1968, -0.3213,  ..., -0.9042, -0.6671, -0.6653],
         [ 0.9280,  0.2896,  0.7292,  ..., -1.0264, -0.7254, -0.9312],
         [ 0.4011,  0.4500,  1.0652,  ..., -1.2439, -0.3919, -0.6764],
         [ 0.5712,  1.1084,  0.7315,  ..., -0.8817, -1.1315, -1.0928]]],
       grad_fn=<UnsafeViewBackward0>), metadata='concat', illegal_output=False)